In [3]:
'''
Midcap Growth Index – Rules-Based Replication (Development)

'''

import pandas as pd
import numpy as np

'''  
pulling cleaned data into the notebook 
'''
df = pd.read_csv("bloomberg_data_cleaned.csv")

#set date to datetime object 
df["date"] = pd.to_datetime(df["date"])
df.set_index(["date","ticker"],inplace= True)
pd.set_option('display.float_format', '{:,.4f}'.format)
df.head()

dates = df.index.get_level_values("date").unique()

In [4]:
'''
Midcap Selection Logic
'''

def select_midcap(snapshot, top_n=1000, midcap_start=200):
    snap = snapshot.dropna(subset=["market_cap"]).copy()
    snap = snap.sort_values("market_cap", ascending=False)
    snap = snap.iloc[:top_n]
    midcap = snap.iloc[midcap_start:]
    return midcap


test_date = dates[0]
snapshot = df.xs(test_date, level="date")
midcap = select_midcap(snapshot)

len(midcap), midcap["market_cap"].median()

(742, np.float64(12125.1901))

In [ ]:
'''
Computing Growth Probability

Since right now we only have P/B data, we can approximate growth
classification using inverse price-to-book (B/P) mapped into a 
smooth probability via a logistic function.
'''

def compute_growth_probability(midcap, k=5.0):
    pb = midcap["price_to_book"].astype(float)

    #convert P/B to B/P since Russell uses B/P
    bp = np.where((pb > 0 ) & np.isfinite(pb), 1.0 / pb, np.nan)
    bp = pd.Series(bp, index=midcap.index)
    bp = bp.fillna(bp.median()) # to fill empty values (for later computation)

    z = (bp -bp.mean()) / (bp.std(ddof=0) + 1e-12) # z-score for standardization
    z_growth = -z # low B/P -> growth, so now high z_growth -> more growthlike

    #normalizing
    #very neg z_growth -> 0 (value), very pos z_growth -> 1 (growth)
    p = 1 / (1 + np.exp(-k * z_growth)) 

    return pd.Series(p, index=midcap.index, name="p_growth")


p_growth = compute_growth_probability(midcap)
p_growth.describe()



count   742.0000
mean      0.6029
std       0.3870
min       0.0000
25%       0.1676
50%       0.7847
75%       0.9594
max       0.9945
Name: p_growth, dtype: float64

In [10]:
'''
Compute index weights
'''
def compute_index_weights(midcap, p_growth):
    cap = midcap["market_cap"].astype(float).clip(lower=0)
    w_cap = cap / cap.sum()

    w = w_cap * p_growth
    w - w / w.sum()

    return pd.Series(w, index=midcap.index, name="weight")

weights = compute_index_weights(midcap, p_growth)
weights.head(), weights.sum()
weights.sort_values(ascending=False).head(10).sum()

np.float64(0.03508200227274532)

In [ ]:
'''
Compute Daily Returns
'''
def compute_price_returns(panel):
    